In [6]:
import numpy as np
import pickle
from astropy.io import fits
from numpy.polynomial import Polynomial
import matplotlib.pyplot as plt
from galpy.util import bovy_conversion, bovy_coords, save_pickles, bovy_plot
from galpy.potential import MWPotential2014, turn_physical_off, vcirc
import astropy.units as u
%pylab inline
from galpy.orbit import Orbit
from scipy import ndimage, signal, interpolate
import gd1_util
import GMC_util

ro=8.
vo=220.


Populating the interactive namespace from numpy and matplotlib


### Test whether very old impacts are important. To do this, we will impact a mock GD-1 stream by subhalos at ~7 Gyr and compute the density today.

In [ ]:
ro=8.
vo=220.

def parse_times(times,age,ro,vo):
    if 'sampling' in times:
        nsam= int(times.split('sampling')[0])
        return [float(ti)/bovy_conversion.time_in_Gyr(vo,ro)
                for ti in numpy.arange(1,nsam+1)/(nsam+1.)*age]
    return [float(ti)/bovy_conversion.time_in_Gyr(vo,ro)
            for ti in times.split(',')]
            
timpacts= parse_times('6.,6.5,7.,7.5,8.,8.5',9.,ro=ro,vo=vo)

print (np.array(timpacts)*bovy_conversion.time_in_Gyr(vo,ro))

In [ ]:
pepperfilename='GD1_oldimpactstest_MW2014_leadingarm.pkl'
sdf_pepper= gd1_util.setup_gd1model(timpact=timpacts)
save_pickles(pepperfilename,sdf_pepper)

In [2]:
with open('GD1_oldimpactstest_MW2014_leadingarm.pkl','rb') as savefile:
            #sdf_smooth= pickle.load(savefile,encoding='latin1')
            sdf_pepper= pickle.load(savefile,encoding='latin1')

In [3]:
sdf_smooth=gd1_util.setup_gd1model()

In [4]:
apar_full,x_full,y_full,z_full,vx_full,vy_full,vz_full = GMC_util.aparxv_stream(sdf_smooth,sdf_pepper)


In [7]:
print (sdf_pepper._timpact*bovy_conversion.time_in_Gyr(vo,ro))

[6.  6.5 7.  7.5 8.  8.5]


In [8]:
# randomly select an apar and its velocity
apar_imp=apar_full[4][10]
v_imp=np.c_[vx_full[4][10],vy_full[4][10],vz_full[4][10]]
M_imp=10**9./bovy_conversion.mass_in_msol(vo,ro)
timp=sdf_pepper._timpact[4]

def rs(M):
    return 1.05*(M/10**8.)**(0.5)

rs_imp=rs(10**9.)/8.

print (rs_imp)

print (v_imp)

0.4150489428970998
[[ 0.63723023 -0.43195544  0.27582483]]


In [9]:
#import streampepperdf

sdf_pepper.set_impacts(impactb=[0.1/8.],subhalovel=v_imp,impact_angle=[apar_imp],timpact=[timp],rs=[rs_imp],GM=[M_imp])



/home/nil/anaconda3/envs/galpy1.4/lib/python3.6/site-packages/galpy/df/streamgapdf.py:1227: RuntimeWarning: divide by zero encountered in double_scalars
  b0 *= b/numpy.sqrt(numpy.sum(b0**2))

/home/nil/anaconda3/envs/galpy1.4/lib/python3.6/site-packages/galpy/df/streamgapdf.py:1227: RuntimeWarning: invalid value encountered in multiply
  b0 *= b/numpy.sqrt(numpy.sum(b0**2))



In [10]:
apar=np.linspace(0.03,1.14,100)

dens_unp=[]
dens=[]
omega_unp=[]
omega=[]

for a in apar :
    dens_unp.append(sdf_smooth._density_par(a))
    omega_unp.append(sdf_smooth.meanOmega(a,oned=True))
    dens.append(sdf_pepper._densityAndOmega_par_approx(a)[0])
    omega.append(sdf_pepper._densityAndOmega_par_approx(a)[1])
    

dens_unp=np.array(dens_unp)
dens=np.array(dens)
omega=np.array(omega)

/mnt/d/Dropbox/GD-1_data/Analyze_GD-1/streampepperdf_new.py:793: RuntimeWarning: invalid value encountered in greater_equal
  lowbindx= numpy.argmax((lowx >= 0.)*(nlowx <= 10.)) # not too crazy

/mnt/d/Dropbox/GD-1_data/Analyze_GD-1/streampepperdf_new.py:793: RuntimeWarning: invalid value encountered in less_equal
  lowbindx= numpy.argmax((lowx >= 0.)*(nlowx <= 10.)) # not too crazy



In [11]:
print (dens)

[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]


In [ ]:
def convert_dens_to_obs(apars,dens,dens_smooth,mO,poly_deg=3,poly_norm=True):
    """
    NAME:
        convert_dens_to_obs
    PURPOSE:
        Convert track to observed coordinates
    INPUT:
        apars - parallel angles
        dens - density(apars)
        dens_smooth - smooth-stream density(apars)
        mO= (None) mean parallel frequency (1D) 
            [needs to be set to get density on same grid as track]
        poly_deg= (3) degree of the polynomial to fit for the 'smooth' stream
    OUTPUT:
        (xi,dens/smooth)
    """
    mT= sdf_pepper.meanTrack(apars,_mO=mO,coord='lb')
    outll= numpy.linspace(numpy.amin(mT[0]),numpy.amax(mT[0]),len(apars))
    #phi12=lb_to_phi12(mT[0],mT[1],degree=True)
    #print (phi12[:,0])
    
    pp= Polynomial.fit(apars,dens,deg=3)
    outphi1= numpy.arange(0.25,14.35,0.1)
    # Interpolate density
    ipll= interpolate.InterpolatedUnivariateSpline(mT[0],apars)
    ipdens= interpolate.InterpolatedUnivariateSpline(apars,dens/dens_smooth)
    return (outll,ipdens(ipll(outll)))

ll,dd=convert_dens_to_obs(apar,dens,dens_unp,omega)
    

In [ ]:
plt.figure()
plt.plot(ll,dd)
plt.show()